In [1]:
import os
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import cv2
from keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler
%matplotlib inline


Loading the images 

In [2]:
BASE_PATH = "../input/images/Images/"

LABELS = set()

paths = []

for d in os.listdir(BASE_PATH):
    LABELS.add(d)
    paths.append((BASE_PATH+d, d))

In [3]:
# VGG16 use 224x224 RGB image -> convert image 
def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (224,224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [4]:
X = []
y = []
for path, label in paths:
    for image_path in os.listdir(path):
        image = load_and_preprocess_image(path+"/"+image_path)
        
        X.append(image)
        y.append(label)


In [5]:
# encoding labels
encoder = LabelBinarizer()
 
X = np.array(X)

y = encoder.fit_transform(np.array(y))

Training and evaluating 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
base_model = VGG16(weights='imagenet', include_top=False)

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation='relu')(x)
preds = layers.Dense(len(LABELS), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=preds)

# set 5 last layer can config
for layer in model.layers[:-5]:
    layer.trainable=False
for layer in model.layers[-5:]:
    layer.trainable=True



In [8]:
model.compile("adam", loss='categorical_crossentropy', metrics=["accuracy"])

print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [9]:
early_stopping = EarlyStopping(patience=5, verbose=1,restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=3,verbose=1)

In [10]:
# augs_gen = ImageDataGenerator(featurewise_center=False, samplewise_center=False,
#                               featurewise_std_normalization=False, samplewise_std_normalization=False,
#                               zca_whitening=False, rotation_range=10, zoom_range=0.1, width_shift_range=0.2,
#                               height_shift_range=0.2, horizontal_flip=True, vertical_flip=False)
# augs_gen.fit(X_train)

In [11]:
h = model.fit(X_train,y_train,batch_size=64,epochs=20,validation_data=(X_test,y_test), callbacks=[early_stopping, reduce_lr])

Epoch 1/20
 44/226 [====>.........................] - ETA: 13:49 - loss: 5.2408 - accuracy: 0.0401

: 

: 

In [ ]:
model_score = model.evaluate(X_test,y_test)
print("Model Test Loss:",model_score[0])
print("Model Test Accuracy:",model_score[1])

model_json = model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)
    
model.save("model.h5")
print("Weights Saved")

NameError: name 'model' is not defined